# Train Model

### Imports

In [ ]:
import sys
from config import Config
sys.path.append(Config.root_path)

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
# Load Model
from Python.model.segmentation_model import deeplabv3plus
# Load Dataset & Preprocessing
from Python.data_processing.utils import get_png_paths_from_dir
from Python.data_processing.utils import load_weight_map
from Python.data_processing.data_generator import generate_image_dataset_from_files
from Python.data_processing.data_generator import split_dataset
from Python.data_processing.data_generator import augment_dataset
# Print Model Prediction
from Python.data_processing.compare_predictions import show_predictions
# Save Model
from Python.data_processing.save_model import write_model_to_disk


print(tf.config.list_physical_devices('GPU'))

### Prepare dataset

In [ ]:
image_files = get_png_paths_from_dir(Config.image_path)
mask_files = get_png_paths_from_dir(Config.segmentation_path)
weight_map = load_weight_map(Config.weight_map_path)
weights = np.zeros(Config.output_channels, dtype=float)
for key in weight_map.keys():
    weights[int(key)] = weight_map.get(key)

dataset = generate_image_dataset_from_files(
    image_files, 
    mask_files, 
    Config.batch_size,
    tf.data.AUTOTUNE, 
    Config.shuffle_size, 
    weights
)
train_dataset, val_dataset, test_dataset = split_dataset(
    dataset, Config.train_size, Config.val_size, Config.test_size
)
train_dataset = augment_dataset(train_dataset)

### Load Model

In [ ]:
model = deeplabv3plus(
    Config.input_shape,
    Config.batch_size,
    Config.output_channels,
    Config.channels_low,
    Config.channels_high,
    Config.middle_repeat
)
model.compile(
    optimizer = 'adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = ["accuracy"]
)

### Compare Image, Segmentation Mask and Prediction

In [ ]:
show_predictions(model, dataset)

### Model Training

Train Model

In [ ]:
model_history = model.fit(
    train_dataset,
    epochs=20,
    validation_data=val_dataset,
    shuffle=True
) 

Show Model Performance

In [ ]:
loss = model_history.history['loss']
val_loss = model_history.history['val_loss']

plt.figure()
plt.plot(model_history.epoch, loss, 'r', label='Training loss')
plt.plot(model_history.epoch, val_loss, 'bo', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss Value')
plt.ylim([0, 1])
plt.legend()
plt.show()

### Save Model

In [ ]:
write_model_to_disk(model, model_history, Config)